In [41]:
import torch
from gensim.models import KeyedVectors
import pyprob #https://github.com/probprog/pyprob #!pip install pyprob
from pyprob import Model 
from pyprob.distributions import Categorical, Uniform, Normal
from pyprob import InferenceEngine
import matplotlib.pyplot as plt

In [12]:
wv = KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300.bin", binary=True)

In [2]:
# define global question template variables
color_names = ["red", "green", "blue", "orange", "gray", "yellow"]
non_relational_qs = [
    "What shape is the {} object?",
    "Is the {} object on the left?",
    "Is the {} object on the top?"
]
relational_qs = [
    "What shape is the object closest to the {} object?",
    "What shape is the object furthest from the {} object?",
    "How many objects are the same shape as the {} object?"
]
all_qs = [non_relational_qs, relational_qs]

In [10]:
def text_to_vec(text):
    return torch.tensor([wv[w] for w in text.split() if w in wv]).mean(dim=0)

In [235]:
class QuestionsGen(Model):
    def __init__(self, name="QuestionsModel", opt=None):
        super().__init__(name=name)
        self.opt = opt
        self.colors = 6
        
    def compile_question(self, color, qtype, qsubtype, template):
        # compile a question latent
        color_vec = [0, 0, 0, 0, 0, 0]
        type_vec = [0, 0]
        subtype_vec = [0, 0, 0]
        color_vec[color] = 1
        type_vec[qtype] = 1
        subtype_vec[qsubtype] = 1
        question_vec = color_vec + type_vec + subtype_vec
        
        # get the text of the question
        question_text = all_qs[qtype][qsubtype].format(color_names[color])
        
        return question_text, question_vec
        
    def forward(self):
        c_i = pyprob.sample(Categorical(logits=[1 for _ in range(self.colors)]))
        t_i = pyprob.sample(Categorical(logits=(1,1)))
        st_i = pyprob.sample(Categorical(logits=(1,1,1)))
        tmp_i = pyprob.sample(Categorical(logits=(1,1)))
        cand_q_text, cand_q_latent = self.compile_question(c_i, t_i, st_i, tmp_i)
        pyprob.tag(cand_q_latent, name="question_latent")
        pyprob.tag(cand_q_text, name="question_text")
        cand_vec = text_to_vec(cand_q_text)
        
        pyprob.observe(Normal(cand_vec, 0.0001), name="observed_question")
        return cand_q_text, cand_q_latent

In [236]:
model = QuestionsGen(Model)
posterior_dist = model.posterior(
    num_traces=100,
    inference_engine = InferenceEngine.IMPORTANCE_SAMPLING,
    observe = {'observed_question': text_to_vec("What shape is the object closest to the green object?")},
    initial_trace=None
)

Time spent  | Time remain.| Progress             | Trace   | ESS   | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 100/100 |  1.00 | 626.68       


In [237]:
text_to_vec("What shape is the object closest to the green object?")

tensor([ 0.0366,  0.0726,  0.1249,  0.0187, -0.1609,  0.0097,  0.0670, -0.1761,
         0.0833,  0.0596, -0.0161, -0.0872, -0.0521,  0.0668, -0.0579,  0.0642,
         0.0260,  0.0676,  0.0424, -0.0598, -0.0748,  0.0819, -0.0634, -0.0424,
         0.0293,  0.0305, -0.1050,  0.0417,  0.0909, -0.0729,  0.0141, -0.0089,
        -0.0165,  0.0069,  0.0512,  0.0149, -0.0076,  0.0275,  0.0228,  0.0790,
        -0.0027, -0.0387,  0.0713, -0.0168, -0.0284, -0.0194, -0.0288, -0.0363,
         0.0508,  0.0694, -0.0378,  0.0667, -0.0266, -0.0511,  0.0195,  0.0811,
         0.0116, -0.1023,  0.0794, -0.0288, -0.0641,  0.0511, -0.1063, -0.0985,
        -0.0550, -0.0478,  0.0095,  0.0657, -0.0705,  0.0923,  0.0228, -0.0708,
         0.1691, -0.0529, -0.0665, -0.0604,  0.0918,  0.0684, -0.0146,  0.2277,
         0.0756, -0.1240,  0.0429, -0.0115, -0.0397, -0.0699, -0.0526,  0.0286,
         0.0602, -0.0271,  0.0320, -0.0122, -0.0460, -0.0234,  0.0305, -0.0123,
         0.0504,  0.0492, -0.0045,  0.03

In [238]:
posterior_dist.sample()["question_text"]

'What shape is the object closest to the green object?'

In [239]:
posterior_dist.sample()["question_latent"]

[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0]

In [210]:
QG = QuestionsGen()
for _ in range(25):
    print(QG.forward())

('Is the green object on the top?', [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1])
('What shape is the object closest to the gray object?', [0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0])
('What shape is the object closest to the green object?', [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0])
('Is the gray object on the left?', [0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0])
('Is the blue object on the top?', [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1])
('Is the yellow object on the left?', [0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0])
('How many objects are the same shape as the yellow object?', [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1])
('What shape is the object furthest from the blue object?', [0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0])
('What shape is the object furthest from the orange object?', [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0])
('Is the red object on the top?', [1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1])
('What shape is the object closest to the blue object?', [0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0])
('What shape is the object furthest from the red object?', [1, 0, 0, 0, 0, 0, 0,

In [ ]:
QG.question_likelihood([0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1], "Is the blue ")

In [14]:
# generate a bunch of latents to make sure they are 1 with the probability we think they should be
all_latents = []
for _ in range(10000):
    _, latent = QG.forward()
    all_latents.append(latent)
all_latents = torch.tensor(all_latents).float()
print(all_latents.mean(dim=0))

tensor([0.1668, 0.1654, 0.1631, 0.1686, 0.1682, 0.1679, 0.5003, 0.4997, 0.3402,
        0.3272, 0.3326])
